# LEZIONE 13 — Pipeline di Scikit-Learn

## Obiettivi della Lezione

Al termine di questa lezione sarai in grado di:

1. **Comprendere** perché le Pipeline sono essenziali nei progetti reali
2. **Costruire** Pipeline che combinano preprocessing e modello
3. **Usare** ColumnTransformer per gestire feature diverse
4. **Evitare** il data leakage con preprocessing corretto
5. **Strutturare** workflow di ML riproducibili e manutenibili

## Perché è Importante

Le Pipeline risolvono **problemi critici** che ogni data scientist affronta:

- **Data Leakage**: preprocessing fatto prima dello split contamina i risultati
- **Riproducibilità**: senza pipeline, il codice diventa fragile e difficile da mantenere
- **Produzione**: un modello senza pipeline non può essere deployato facilmente

## Posizione nel Percorso

```
Lezione 12: Overfitting (cosa può andare storto)
    ↓
→ Lezione 13: Pipeline (come strutturare il workflow) ← SEI QUI
    ↓
Lezione 14: Modelli Tree-Based (Decision Tree, Random Forest)
```

---

## Librerie Utilizzate

```python
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
```

# Section 1 — Il Problema: Perché Servono le Pipeline?

## Scenario Tipico (SBAGLIATO)

```python
# ❌ ERRORE COMUNE: preprocessing su TUTTI i dati prima dello split

# 1. Carichiamo i dati
X = load_data()

# 2. Preprocessing su TUTTO il dataset
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # ← ERRORE! fit su tutti i dati

# 3. Poi facciamo lo split
X_train, X_test = train_test_split(X_scaled)

# 4. Alleniamo il modello
model.fit(X_train, y_train)
```

## Perché è Sbagliato? DATA LEAKAGE

```
            PRIMA DELLO SPLIT
            ─────────────────
               ┌─────────┐
    Tutti i ─► │ Scaler  │ ─► fit() vede ANCHE i dati di test!
    dati       │ fit()   │
               └─────────┘
                    │
            ┌───────┴───────┐
            ▼               ▼
         Train            Test
         
Il modello "bara": ha visto informazioni sul test set!
```

**Conseguenze:**
- Le performance sul test set sono **ottimistiche**
- In produzione il modello avrà performance **peggiori**
- Hai violato la regola fondamentale del ML

## Scenario Corretto (MA SCOMODO)

```python
# ✅ CORRETTO ma verboso e error-prone

# 1. Prima lo split
X_train, X_test = train_test_split(X)

# 2. Fit SOLO su train
scaler = StandardScaler()
scaler.fit(X_train)  # ← fit solo su train

# 3. Transform su entrambi
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. Stessa cosa per ogni step di preprocessing...
imputer = SimpleImputer()
imputer.fit(X_train_scaled)
X_train_imputed = imputer.transform(X_train_scaled)
X_test_imputed = imputer.transform(X_test_scaled)

# 5. Allena modello
model.fit(X_train_imputed, y_train)

# 6. Per predizioni su nuovi dati: ripeti tutti i transform!
```

**Problemi:**
- Codice lungo e ripetitivo
- Facile dimenticare un passaggio
- Difficile da mantenere
- Impossibile da usare in produzione

# Section 2 — La Soluzione: sklearn.pipeline.Pipeline

## Cos'è una Pipeline?

Una **Pipeline** è una sequenza ordinata di trasformazioni + un modello finale, incapsulati in un unico oggetto.

```
               PIPELINE
    ┌─────────────────────────────────┐
    │                                 │
    │  Step 1      Step 2      Step 3 │
    │ ┌──────┐   ┌──────┐   ┌───────┐ │
X ─►│ │Imputer│─► │Scaler│ ─►│ Model │ │─► y_pred
    │ └──────┘   └──────┘   └───────┘ │
    │                                 │
    └─────────────────────────────────┘
    
    pipe.fit(X_train, y_train)  # allena TUTTO
    pipe.predict(X_test)        # applica TUTTO
```

## Vantaggi

| Senza Pipeline | Con Pipeline |
|----------------|--------------|
| Codice frammentato | Oggetto unico |
| Facile dimenticare step | Tutti gli step garantiti |
| Data leakage possibile | Data leakage impossibile |
| Difficile salvare/caricare | Un solo file da salvare |
| Cross-validation manuale | CV automatica e corretta |

## Sintassi Base

```python
from sklearn.pipeline import Pipeline

# Metodo 1: esplicito con nomi
pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression())
])

# Metodo 2: senza nomi (make_pipeline)
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler(),
    LogisticRegression()
)
```

## Come Funziona

Quando chiami `pipe.fit(X, y)`:
1. Ogni transformer chiama `fit_transform()` sui dati
2. L'output viene passato al passo successivo
3. L'ultimo step (modello) chiama solo `fit()`

Quando chiami `pipe.predict(X)`:
1. Ogni transformer chiama solo `transform()` (NO fit!)
2. L'ultimo step chiama `predict()`

```
fit(X_train, y_train):
─────────────────────
X_train → [Imputer.fit_transform] → [Scaler.fit_transform] → [Model.fit]

predict(X_test):
────────────────
X_test → [Imputer.transform] → [Scaler.transform] → [Model.predict] → y_pred
```

In [2]:
# Dimostrazione: Prima Pipeline

import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.datasets import make_classification

# Creiamo un dataset con valori mancanti
np.random.seed(42)
X, y = make_classification(
    n_samples=500,
    n_features=10,
    n_informative=6,
    flip_y=0.1,
    random_state=42
)

# Aggiungiamo valori mancanti (~10%)
mask = np.random.random(X.shape) < 0.10
X[mask] = np.nan

print("Dataset con valori mancanti:")
print(f"Shape: {X.shape}")
print(f"Valori mancanti: {np.isnan(X).sum()} ({np.isnan(X).mean()*100:.1f}%)")

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

# ============================================================================
# METODO 1: Senza Pipeline (per confronto)
# ============================================================================
print("\n" + "="*60)
print("METODO 1: Senza Pipeline")
print("="*60)

# Step 1: Imputer (fit solo su train)
imputer = SimpleImputer(strategy='mean')
imputer.fit(X_train)
X_train_imputed = imputer.transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Step 2: Scaler (fit solo su train)
scaler = StandardScaler()
scaler.fit(X_train_imputed)
X_train_scaled = scaler.transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# Step 3: Model
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)
score_no_pipe = model.score(X_test_scaled, y_test)

print(f"Test Accuracy: {score_no_pipe:.4f}")
print("Codice: ~15 righe, 6 variabili intermedie")

# ============================================================================
# METODO 2: Con Pipeline
# ============================================================================
print("\n" + "="*60)
print("METODO 2: Con Pipeline")
print("="*60)

# Tutto in un oggetto!
pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(max_iter=1000))
])

# Fit e score in due righe
pipe.fit(X_train, y_train)
score_pipe = pipe.score(X_test, y_test)

print(f"Test Accuracy: {score_pipe:.4f}")
print("Codice: 2 righe, 0 variabili intermedie")

# Verifica che i risultati siano identici
print(f"\nRisultati identici? {np.isclose(score_no_pipe, score_pipe)}")

# ============================================================================
# Cross-Validation con Pipeline
# ============================================================================
print("\n" + "="*60)
print("Cross-Validation con Pipeline")
print("="*60)

# Con pipeline, la CV è automaticamente corretta
# Ogni fold fa fit del preprocessing solo sui dati di train di quel fold
cv_scores = cross_val_score(pipe, X, y, cv=5, scoring='accuracy')

print(f"CV Scores: {[f'{s:.3f}' for s in cv_scores]}")
print(f"CV Mean: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
print("\n✅ Nessun data leakage: il preprocessing è rifatto ad ogni fold!")

Dataset con valori mancanti:
Shape: (500, 10)
Valori mancanti: 525 (10.5%)

METODO 1: Senza Pipeline
Test Accuracy: 0.7760
Codice: ~15 righe, 6 variabili intermedie

METODO 2: Con Pipeline
Test Accuracy: 0.7760
Codice: 2 righe, 0 variabili intermedie

Risultati identici? True

Cross-Validation con Pipeline
CV Scores: ['0.760', '0.690', '0.730', '0.830', '0.770']
CV Mean: 0.7560 ± 0.0463

✅ Nessun data leakage: il preprocessing è rifatto ad ogni fold!


# Section 3 — ColumnTransformer: Pipeline per Dati Eterogenei

## Il Problema: Colonne Diverse, Trattamenti Diversi

Nei dataset reali abbiamo spesso:
- **Colonne numeriche** → SimpleImputer(mean) + StandardScaler
- **Colonne categoriche** → SimpleImputer(most_frequent) + OneHotEncoder
- **Colonne ordinali** → OrdinalEncoder
- **Colonne da ignorare** → passthrough o drop

## La Soluzione: `sklearn.compose.ColumnTransformer`

```python
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer([
    ('name_1', transformer_1, columns_1),
    ('name_2', transformer_2, columns_2),
    ...
], remainder='drop')  # cosa fare con le altre colonne
```

### Parametri importanti:

| Parametro | Opzioni | Descrizione |
|-----------|---------|-------------|
| `remainder` | `'drop'` (default) | Scarta le colonne non specificate |
| | `'passthrough'` | Mantiene le colonne non trasformate |
| | transformer | Applica un transformer alle colonne rimanenti |

### Specificare le colonne:

```python
# Per nome (con DataFrame)
columns = ['age', 'income', 'score']

# Per indice (con array)
columns = [0, 2, 5]

# Con selector (sklearn >= 1.0)
from sklearn.compose import make_column_selector

num_cols = make_column_selector(dtype_include=np.number)
cat_cols = make_column_selector(dtype_include=object)
```

## Pattern Comune: Pipeline Completa

```
┌───────────────────────────────────────────────────────────┐
│                    ColumnTransformer                       │
│  ┌─────────────────┐    ┌─────────────────┐               │
│  │ numeric_pipe    │    │ categorical_pipe │               │
│  │  Imputer(mean)  │    │  Imputer(freq)   │               │
│  │  StandardScaler │    │  OneHotEncoder   │               │
│  └────────┬────────┘    └────────┬─────────┘               │
│           │                       │                         │
│           └───────────┬───────────┘                         │
│                       │ concatena                           │
│                       ▼                                     │
│            [features trasformate]                           │
└───────────────────────────────────────────────────────────┘
                        │
                        ▼
               ┌─────────────────┐
               │     Modello     │
               │ (es. XGBoost)   │
               └─────────────────┘
```

In [3]:
# Dimostrazione: ColumnTransformer con dati misti

from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.ensemble import RandomForestClassifier

# Creiamo un dataset realistico con dati misti
np.random.seed(42)
n_samples = 500

df = pd.DataFrame({
    'age': np.random.randint(18, 70, n_samples).astype(float),
    'income': np.random.normal(50000, 20000, n_samples),
    'credit_score': np.random.randint(300, 850, n_samples).astype(float),
    'employment': np.random.choice(['employed', 'self-employed', 'unemployed', 'retired'], n_samples),
    'education': np.random.choice(['high_school', 'bachelor', 'master', 'phd'], n_samples),
    'home_owner': np.random.choice(['yes', 'no', 'rent'], n_samples)
})

# Target binario
df['approved'] = ((df['income'] > 40000) & 
                   (df['credit_score'] > 600)).astype(int)

# Aggiungiamo valori mancanti realistici
df.loc[np.random.choice(n_samples, 30, replace=False), 'income'] = np.nan
df.loc[np.random.choice(n_samples, 20, replace=False), 'employment'] = np.nan
df.loc[np.random.choice(n_samples, 15, replace=False), 'credit_score'] = np.nan

print("Dataset con dati misti:")
print(df.head(10))
print(f"\nDtypes:\n{df.dtypes}")
print(f"\nValori mancanti:\n{df.isnull().sum()}")

# ============================================================================
# Definiamo le colonne
# ============================================================================
numeric_features = ['age', 'income', 'credit_score']
categorical_features = ['employment', 'education', 'home_owner']

# Pipeline per features numeriche
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),  # median per outlier
    ('scaler', StandardScaler())
])

# Pipeline per features categoriche
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# ColumnTransformer che combina tutto
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# Pipeline finale con modello
full_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

print("\n" + "="*60)
print("Struttura della Pipeline:")
print("="*60)
print(full_pipeline)

# ============================================================================
# Training e Valutazione
# ============================================================================
X = df.drop('approved', axis=1)
y = df['approved']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

# Fit e predict in una riga ciascuno!
full_pipeline.fit(X_train, y_train)
score = full_pipeline.score(X_test, y_test)

print(f"\n{'='*60}")
print("Risultati:")
print(f"{'='*60}")
print(f"Test Accuracy: {score:.4f}")

# Cross-validation con la pipeline completa
cv_scores = cross_val_score(full_pipeline, X, y, cv=5)
print(f"CV Accuracy: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")

# ============================================================================
# Ispezionare la Pipeline
# ============================================================================
print(f"\n{'='*60}")
print("Ispezione della Pipeline:")
print(f"{'='*60}")

# Accedere ai componenti
print("\nAccesso ai componenti:")
print(f"Preprocessor: {full_pipeline['preprocessor']}")
print(f"Classifier: {full_pipeline['classifier']}")

# Feature names dopo one-hot encoding
feature_names = full_pipeline['preprocessor'].get_feature_names_out()
print(f"\nFeatures trasformate ({len(feature_names)} totali):")
print(feature_names[:5], "...")

Dataset con dati misti:
    age        income  credit_score     employment    education home_owner  \
0  56.0  92443.123940         412.0     unemployed       master        yes   
1  69.0  70649.305211         486.0            NaN     bachelor         no   
2  46.0  19612.600681         526.0        retired     bachelor       rent   
3  32.0  40315.318543         609.0     unemployed          phd         no   
4  60.0  75338.222984         397.0     unemployed     bachelor         no   
5  25.0  35846.610688         354.0  self-employed          phd         no   
6  38.0  58876.388563         705.0     unemployed     bachelor         no   
7  56.0  65492.681069         368.0        retired  high_school       rent   
8  36.0  31461.390568         611.0  self-employed     bachelor       rent   
9  40.0  48809.492879         588.0        retired  high_school       rent   

   approved  
0         0  
1         0  
2         0  
3         1  
4         0  
5         0  
6         1  
7    

# Section 4 — Schema Mentale: Quando Usare le Pipeline

## Checklist Decisionale

### ✅ USA Pipeline quando:

| Situazione | Motivo |
|------------|--------|
| Hai **più step** di preprocessing | Eviti variabili intermedie e errori |
| Fai **cross-validation** | Previeni data leakage automaticamente |
| Vuoi fare **hyperparameter tuning** | GridSearchCV lavora con pipeline |
| Il modello va in **produzione** | Un solo oggetto da serializzare |
| Lavori in **team** | Codice più leggibile e manutenibile |

### ⚠️ Pipeline NON necessaria quando:

| Situazione | Alternativa |
|------------|-------------|
| Un solo step di preprocessing | Può bastare il transformer singolo |
| Esplorazione dati (EDA) | Lavori step-by-step per capire |
| Debug di preprocessing | Isoli il problema senza pipeline |

## Regola d'Oro

> **"Se il tuo preprocessing ha più di uno step E farai training/prediction, usa Pipeline"**

## Vantaggi Riassunti

```
SENZA PIPELINE                    CON PIPELINE
─────────────────────────────     ─────────────────────────────
• 6 variabili intermedie          • 0 variabili intermedie
• Rischio errori fit/transform    • Impossibile sbagliare
• Data leakage in CV             • CV automaticamente corretta
• 3 oggetti da salvare           • 1 oggetto da salvare
• Codice lungo                    • Codice compatto
```

## Pattern Tipici da Ricordare

### 1. Pipeline Semplice (solo numeriche)
```python
pipe = make_pipeline(
    SimpleImputer(),
    StandardScaler(),
    LogisticRegression()
)
```

### 2. Pipeline con Dati Misti
```python
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, num_cols),
    ('cat', categorical_pipeline, cat_cols)
])

full_pipe = Pipeline([
    ('prep', preprocessor),
    ('model', XGBClassifier())
])
```

### 3. Pipeline per Produzione
```python
import joblib

# Salva tutto in un file
joblib.dump(full_pipe, 'model.pkl')

# Carica e predici
loaded_pipe = joblib.load('model.pkl')
predictions = loaded_pipe.predict(new_data)
```

# Section 5 — Esercizi Svolti

---

## 🎯 Esercizio 13.1 — Pipeline Base per Dataset Numerico

**Obiettivo**: Creare una pipeline per classificazione con preprocessing numerico.

**Consegna**:
1. Genera un dataset di classificazione con `make_classification` (500 campioni, 8 features)
2. Aggiungi 10% di valori mancanti
3. Crea una pipeline con: SimpleImputer → StandardScaler → SVC
4. Valida con cross-validation a 5 fold
5. Mostra i risultati

In [4]:
# ============================================================================
# SOLUZIONE ESERCIZIO 13.1 — Pipeline Base per Dataset Numerico
# ============================================================================

from sklearn.datasets import make_classification
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
import numpy as np

print("="*60)
print("ESERCIZIO 13.1 — Pipeline Base per Dataset Numerico")
print("="*60)

# 1. Genera dataset
np.random.seed(42)
X, y = make_classification(
    n_samples=500,
    n_features=8,
    n_informative=5,
    n_redundant=2,
    random_state=42
)

print(f"\n1. Dataset generato: {X.shape}")

# 2. Aggiungi 10% valori mancanti
mask = np.random.random(X.shape) < 0.10
X[mask] = np.nan

n_missing = np.isnan(X).sum()
print(f"2. Valori mancanti aggiunti: {n_missing} ({n_missing/(X.shape[0]*X.shape[1])*100:.1f}%)")

# 3. Crea pipeline
pipe = make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler(),
    SVC(kernel='rbf', random_state=42)
)

print(f"\n3. Pipeline creata:")
print(pipe)

# 4. Cross-validation
cv_scores = cross_val_score(pipe, X, y, cv=5, scoring='accuracy')

# 5. Risultati
print(f"\n4-5. Risultati Cross-Validation:")
print(f"     Scores per fold: {[f'{s:.3f}' for s in cv_scores]}")
print(f"     Mean Accuracy: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")

print("\n✅ Pipeline funzionante con preprocessing automatico!")

ESERCIZIO 13.1 — Pipeline Base per Dataset Numerico

1. Dataset generato: (500, 8)
2. Valori mancanti aggiunti: 426 (10.7%)

3. Pipeline creata:
Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler()),
                ('svc', SVC(random_state=42))])

4-5. Risultati Cross-Validation:
     Scores per fold: ['0.930', '0.930', '0.940', '0.890', '0.910']
     Mean Accuracy: 0.9200 ± 0.0179

✅ Pipeline funzionante con preprocessing automatico!


---

## 🎯 Esercizio 13.2 — Pipeline con ColumnTransformer

**Obiettivo**: Costruire una pipeline completa per dati misti (numerici e categorici).

**Consegna**:
1. Usa il Titanic dataset (o creane uno simile)
2. Identifica colonne numeriche e categoriche
3. Crea un ColumnTransformer con:
   - Numeriche: imputer(median) → scaler
   - Categoriche: imputer(most_frequent) → one-hot
4. Aggiungi un classificatore (RandomForest)
5. Valuta con train/test split e mostra accuracy

In [5]:
# ============================================================================
# SOLUZIONE ESERCIZIO 13.2 — Pipeline con ColumnTransformer
# ============================================================================

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

print("="*60)
print("ESERCIZIO 13.2 — Pipeline con ColumnTransformer")
print("="*60)

# 1. Creiamo un dataset simil-Titanic
np.random.seed(42)
n = 600

titanic_like = pd.DataFrame({
    'Pclass': np.random.choice([1, 2, 3], n, p=[0.2, 0.3, 0.5]),
    'Sex': np.random.choice(['male', 'female'], n),
    'Age': np.random.normal(30, 12, n).clip(1, 80),
    'SibSp': np.random.choice([0, 1, 2, 3], n, p=[0.6, 0.25, 0.10, 0.05]),
    'Fare': np.abs(np.random.normal(50, 30, n)),
    'Embarked': np.random.choice(['S', 'C', 'Q'], n, p=[0.7, 0.2, 0.1])
})

# Target: sopravvivenza (correlata a classe, sesso, età)
survival_prob = (
    0.3 
    + 0.2 * (titanic_like['Sex'] == 'female')
    + 0.15 * (titanic_like['Pclass'] == 1)
    - 0.01 * titanic_like['Age']
)
titanic_like['Survived'] = (np.random.random(n) < survival_prob.clip(0, 1)).astype(int)

# Aggiungi valori mancanti realistici
titanic_like.loc[np.random.choice(n, 80, replace=False), 'Age'] = np.nan
titanic_like.loc[np.random.choice(n, 15, replace=False), 'Embarked'] = np.nan

print(f"\n1. Dataset creato: {titanic_like.shape}")
print(titanic_like.head())
print(f"\nValori mancanti:\n{titanic_like.isnull().sum()}")

# 2. Identificare colonne
numeric_cols = ['Age', 'Fare', 'SibSp', 'Pclass']
categorical_cols = ['Sex', 'Embarked']

print(f"\n2. Colonne identificate:")
print(f"   Numeriche: {numeric_cols}")
print(f"   Categoriche: {categorical_cols}")

# 3. ColumnTransformer
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_cols),
    ('cat', categorical_transformer, categorical_cols)
])

print(f"\n3. ColumnTransformer creato")

# 4. Pipeline completa con RandomForest
full_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

print(f"4. Pipeline completa con RandomForest")

# 5. Train/test split e valutazione
X = titanic_like.drop('Survived', axis=1)
y = titanic_like['Survived']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

full_pipeline.fit(X_train, y_train)
accuracy = full_pipeline.score(X_test, y_test)

print(f"\n5. Risultati:")
print(f"   Training samples: {len(X_train)}")
print(f"   Test samples: {len(X_test)}")
print(f"   Test Accuracy: {accuracy:.4f}")

# Bonus: feature names
feature_names = preprocessor.get_feature_names_out()
print(f"\n   Feature trasformate ({len(feature_names)}):")
for i, name in enumerate(feature_names):
    print(f"   {i+1}. {name}")

print("\n✅ Pipeline con dati misti completata!")

ESERCIZIO 13.2 — Pipeline con ColumnTransformer

1. Dataset creato: (600, 7)
   Pclass     Sex        Age  SibSp        Fare Embarked  Survived
0       2    male  33.913596      0   42.618125        C         0
1       3  female  14.986637      2   24.702602        S         0
2       3  female  41.088324      2  115.128282        S         0
3       3  female  27.781174      1   44.723426        C         1
4       1    male  23.727324      1   53.696144        S         0

Valori mancanti:
Pclass       0
Sex          0
Age         80
SibSp        0
Fare         0
Embarked    15
Survived     0
dtype: int64

2. Colonne identificate:
   Numeriche: ['Age', 'Fare', 'SibSp', 'Pclass']
   Categoriche: ['Sex', 'Embarked']

3. ColumnTransformer creato
4. Pipeline completa con RandomForest

5. Risultati:
   Training samples: 450
   Test samples: 150
   Test Accuracy: 0.8200

   Feature trasformate (9):
   1. num__Age
   2. num__Fare
   3. num__SibSp
   4. num__Pclass
   5. cat__Sex_female
   6

---

## 🎯 Esercizio 13.3 — Pipeline con GridSearchCV

**Obiettivo**: Usare GridSearchCV per ottimizzare una pipeline completa.

**Consegna**:
1. Crea una pipeline: StandardScaler → SVC
2. Definisci una griglia di iperparametri per SVC (C, kernel)
3. Usa GridSearchCV con 5-fold CV
4. Mostra i migliori parametri e score
5. Valuta sul test set

In [6]:
# ============================================================================
# SOLUZIONE ESERCIZIO 13.3 — Pipeline con GridSearchCV
# ============================================================================

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.datasets import make_classification

print("="*60)
print("ESERCIZIO 13.3 — Pipeline con GridSearchCV")
print("="*60)

# Dataset
np.random.seed(42)
X, y = make_classification(
    n_samples=400,
    n_features=10,
    n_informative=6,
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

print(f"Dataset: {X.shape}")
print(f"Train: {X_train.shape}, Test: {X_test.shape}")

# 1. Crea pipeline
pipe = make_pipeline(
    StandardScaler(),
    SVC(random_state=42)
)

print(f"\n1. Pipeline creata:")
print(pipe)

# 2. Griglia iperparametri
# NOTA: per accedere ai parametri usiamo stepname__param
# make_pipeline usa nomi automatici: 'standardscaler', 'svc'
param_grid = {
    'svc__C': [0.1, 1, 10, 100],
    'svc__kernel': ['linear', 'rbf', 'poly'],
    'svc__gamma': ['scale', 'auto']  # solo per rbf/poly
}

print(f"\n2. Griglia iperparametri:")
for param, values in param_grid.items():
    print(f"   {param}: {values}")

# 3. GridSearchCV
grid_search = GridSearchCV(
    pipe,
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,  # usa tutti i core
    verbose=0
)

print(f"\n3. GridSearchCV con 5-fold CV...")
grid_search.fit(X_train, y_train)

# 4. Migliori parametri e score
print(f"\n4. Risultati GridSearchCV:")
print(f"   Miglior CV score: {grid_search.best_score_:.4f}")
print(f"   Migliori parametri:")
for param, value in grid_search.best_params_.items():
    print(f"      {param}: {value}")

# 5. Valutazione su test set
test_score = grid_search.score(X_test, y_test)
print(f"\n5. Test Accuracy (con migliori parametri): {test_score:.4f}")

# Mostra top 5 combinazioni
print(f"\n   Top 5 combinazioni:")
results = pd.DataFrame(grid_search.cv_results_)
results = results.sort_values('rank_test_score')
cols = ['rank_test_score', 'mean_test_score', 'std_test_score', 'params']
print(results[cols].head())

print("\n✅ GridSearchCV ottimizza TUTTA la pipeline automaticamente!")

ESERCIZIO 13.3 — Pipeline con GridSearchCV
Dataset: (400, 10)
Train: (300, 10), Test: (100, 10)

1. Pipeline creata:
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(random_state=42))])

2. Griglia iperparametri:
   svc__C: [0.1, 1, 10, 100]
   svc__kernel: ['linear', 'rbf', 'poly']
   svc__gamma: ['scale', 'auto']

3. GridSearchCV con 5-fold CV...

4. Risultati GridSearchCV:
   Miglior CV score: 0.9200
   Migliori parametri:
      svc__C: 10
      svc__gamma: scale
      svc__kernel: rbf

5. Test Accuracy (con migliori parametri): 0.9000

   Top 5 combinazioni:
    rank_test_score  mean_test_score  std_test_score  \
13                1         0.920000        0.022111   
16                1         0.920000        0.022111   
10                3         0.913333        0.033993   
7                 3         0.913333        0.033993   
19                5         0.910000        0.029059   

                                               params  
13  {

# Conclusione

## 🧠 Bignami — Pipeline di Scikit-Learn

### Concetti Chiave

| Concetto | Descrizione |
|----------|-------------|
| **Data Leakage** | Usare info dal test set durante il training (errore grave!) |
| **Pipeline** | Catena di trasformazioni + modello in un oggetto |
| **ColumnTransformer** | Applica trasformazioni diverse a colonne diverse |
| **make_pipeline** | Shortcut: genera nomi automatici |
| **GridSearchCV** | Cerca parametri ottimali nella pipeline completa |

### Sintassi Essenziale

```python
# Pipeline base
from sklearn.pipeline import Pipeline, make_pipeline

pipe = Pipeline([
    ('step1', Transformer1()),
    ('step2', Transformer2()),
    ('model', Classifier())
])

# Equivalente con make_pipeline
pipe = make_pipeline(Transformer1(), Transformer2(), Classifier())

# ColumnTransformer
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer([
    ('num', num_pipe, num_cols),
    ('cat', cat_pipe, cat_cols)
])

# Accesso parametri per GridSearchCV
param_grid = {'stepname__parameter': [val1, val2]}
```

### Vantaggi delle Pipeline

1. ✅ **Prevengono data leakage** - fit solo su training data
2. ✅ **Codice pulito** - niente variabili intermedie
3. ✅ **Riproducibile** - un oggetto contiene tutto
4. ✅ **Produzione-ready** - save/load con joblib
5. ✅ **Hyperparameter tuning** - GridSearchCV nativo

### Errori da Evitare

| ❌ Errore | ✅ Soluzione |
|-----------|-------------|
| Fit scaler su tutto X | Usare Pipeline con CV |
| Dimenticare handle_unknown | `OneHotEncoder(handle_unknown='ignore')` |
| Variabili intermedie | Tutto in Pipeline |
| Salvare modello separato | `joblib.dump(pipeline, 'model.pkl')` |

---

## 📚 Prossima Lezione

**Lezione 14**: Alberi Decisionali e Random Forest
- Come funziona un Decision Tree
- Gini vs Entropy
- Ensemble: Bagging e Random Forest
- Feature Importance